<a href="https://colab.research.google.com/github/CarloRomeoGitHub/ML-exercises/blob/master/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
import PIL
import skimage as sk
import random
from zipfile import ZipFile
import tensorflow.keras.layers as Layers
import tensorflow.keras.activations as Actications
import tensorflow.keras.models as Models
import tensorflow.keras.optimizers as Optimizer
import tensorflow.keras.metrics as Metrics
import tensorflow.keras.utils as Utils
from keras.utils.vis_utils import model_to_dot
import os
import matplotlib.pyplot as plot
import cv2
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix as CM
from random import randint
from IPython.display import SVG
import matplotlib.gridspec as gridspec

Using TensorFlow backend.


In [0]:
#import shutil

#shutil.rmtree('../root/tensorflow_datasets')

In [0]:
2
3
4
# Create a ZipFile Object and load sample.zip in it
with ZipFile('../content/drive/My Drive/intel-image-classification.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall('../content/drive/My Drive/intel')

In [0]:
def get_images(directory):
    Images = []
    Labels = []  # 0 for Building , 1 for forest, 2 for glacier, 3 for mountain, 4 for Sea , 5 for Street
    label = 0
    
    for labels in os.listdir(directory): #Main Directory where each class label is present as folder name.
        if labels == 'glacier': #Folder contain Glacier Images get the '2' class label.
            label = 2
        elif labels == 'sea':
            label = 4
        elif labels == 'buildings':
            label = 0
        elif labels == 'forest':
            label = 1
        elif labels == 'street':
            label = 5
        elif labels == 'mountain':
            label = 3
        
        for image_file in os.listdir(directory+labels): #Extracting the file name of the image from Class Label folder
            image = cv2.imread(directory+labels+r'/'+image_file) #Reading the image (OpenCV)
            image = cv2.resize(image,(150,150)) #Resize the image, Some images are different sizes. (Resizing is very Important)
            Images.append(image)
            Labels.append(label)
    
    return shuffle(Images,Labels,random_state=817328462) #Shuffle the dataset you just prepared.

def get_classlabel(class_code):
    labels = {2:'glacier', 4:'sea', 0:'buildings', 1:'forest', 5:'street', 3:'mountain'}
    
    return labels[class_code]

In [0]:
Images, Labels = get_images('../content/drive/My Drive/intel/seg_train/seg_train/') #Extract the training images from the folders.

Images_train = np.array(Images) #converting the list of images to numpy array.
Labels_train = np.array(Labels)



In [0]:
Images_test, Labels_test = get_images('../content/drive/My Drive/intel/seg_test/seg_test/') #Extract the training images from the folders.

Images_test = np.array(Images_test) #converting the list of images to numpy array.
Labels_test = np.array(Labels_test)



In [0]:
Images_pred, Labels_pred = get_images('../content/drive/My Drive/intel/seg_pred/seg_pred/') #Extract the training images from the folders.

Images_pred = np.array(Images_pred) #converting the list of images to numpy array.
Labels_pred = np.array(Labels_pred)

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
model = Sequential()
model.add(Conv2D(64, 7, activation = "relu", padding = "same", input_shape = [150, 150, 3]))

#Parametri:
#7x7=49 pesi
#1 bias
#50 parametri; 50 x 64 = 3200

model.add(MaxPooling2D(2))
model.add(Conv2D(128,3, activation = "relu", padding ="same"))
model.add(Conv2D(128,3, activation = "relu", padding ="same")) 
model.add(MaxPooling2D(2))
model.add(Conv2D(256,3, activation = "relu", padding ="same"))
model.add(Conv2D(256,3, activation = "relu", padding ="same"))
model.add(MaxPooling2D(2))
model.add(Flatten())
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.5))

#layer di dropout(attivo solo in fase di addestramento): server per evitare l'overfittng e facilitare il learning.
#Durante l'addestramente inibisce ad ogni passo una percentuale di neuroni, e cerca di far riconoscere gli oggetti usando solo una parte di neuroni
# 0.5 è la percentuale di neuroni che escludo dal task


model.add(Dense(10, activation = "softmax"))

#compilazione modello 
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()





Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 150, 150, 64)      9472      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 75, 75, 128)       73856     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 75, 75, 128)       147584    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 37, 37, 128)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 37, 37, 256)       295168    
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 37, 37, 256)      

In [0]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)

earlystopper = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

In [0]:
model.fit(Images_train, Labels_train, batch_size=50, epochs=30, callbacks=[model_checkpoint, earlystopper])
model.evaluate(Images_test, Labels_test)

Train on 14034 samples
Epoch 1/30
14000/14034 [============================>.] - ETA: 0s - loss: 2.5828 - acc: 0.2799
Epoch 00001: loss did not improve from 0.08498
14034/14034 [==============================] - 47s 3ms/sample - loss: 2.5802 - acc: 0.2802
Epoch 2/30
   50/14034 [..............................] - ETA: 46s - loss: 1.3561 - acc: 0.4600

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


14000/14034 [============================>.] - ETA: 0s - loss: 1.2472 - acc: 0.4902
Epoch 00002: loss did not improve from 0.08498
14034/14034 [==============================] - 47s 3ms/sample - loss: 1.2471 - acc: 0.4903
Epoch 3/30
14000/14034 [============================>.] - ETA: 0s - loss: 1.0828 - acc: 0.5635
Epoch 00003: loss did not improve from 0.08498
14034/14034 [==============================] - 47s 3ms/sample - loss: 1.0825 - acc: 0.5638
Epoch 4/30
  700/14034 [>.............................] - ETA: 44s - loss: 1.1027 - acc: 0.5629